In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
import os
import scipy as sp
import scipy.fftpack
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from scipy.signal import savgol_filter
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]
from sklearn.preprocessing import StandardScaler


In [2]:
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_plate_infos.pick"

time_plate_info = pd.read_pickle(path_save)

path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_plate_infos.pick"

global_hypha_info = pd.read_pickle("global_hypha_info.pick")
time_hypha_info = pd.read_pickle("time_hypha_infos.pick")


In [3]:
global_plate_infos = pd.read_pickle("global_plate_infos.pick")
global_plate_infos = global_plate_infos.set_index('Plate')
global_plate_infos.sort_values(['CrossDate'])['CrossDate']

Plate
94     20201123
80     20201127
91     20201217
152    20201224
102    20201226
26     20210104
10     20210107
76     20210108
790    20210907
88     20210915
792    20210915
787    20210922
800    20210928
808    20210928
785    20210928
799    20211001
Name: CrossDate, dtype: object

In [4]:
global_plate_infos.to_csv("global_plate_infos.csv")
time_hypha_info.to_csv("time_hypha_infos.csv")
time_plate_info.to_csv("time_plate_infos.csv")

global_hypha_info.to_csv("global_hypha_info.csv")

In [5]:
time_hypha_info.columns

Index(['end', 'distance_final_pos', 'timedelta', 'time_since_emergence',
       'speed', 'timestep', 'timestep_init', 'time_init', 'degree',
       'width_tip_edge', 'av_width', 'has_reached_final_pos', 'in_ROI',
       'time_since_begin_exp', 'width_root_edge', 'Plate', 'folder', 't',
       'density_window500', 'density_window1000', 'density_window2000'],
      dtype='object')

In [6]:
time_hypha_info

,end,distance_final_pos,timedelta,time_since_emergence,speed,timestep,timestep_init,time_init,degree,width_tip_edge,...,has_reached_final_pos,in_ROI,time_since_begin_exp,width_root_edge,Plate,folder,t,density_window500,density_window1000,density_window2000
0,0,5273.293468,2.0,0.0,182.893809,0,0,0.0,1,10.99649,...,False,True,0.0,NaN,799,Analysis_1636046826216022929_0_100_Version9,0,NaN,NaN,NaN
1,1,264.461717,2.0,0.0,128.878255,0,0,0.0,1,6.927419,...,False,True,0.0,NaN,799,Analysis_1636046826216022929_0_100_Version9,0,NaN,NaN,NaN
4,4,912.054823,2.0,0.0,83.043454,0,0,0.0,1,7.549542,...,False,True,0.0,NaN,799,Analysis_1636046826216022929_0_100_Version9,0,NaN,NaN,NaN
6,6,150.748134,2.0,0.0,16.606078,0,0,0.0,1,None,...,False,True,0.0,None,799,Analysis_1636046826216022929_0_100_Version9,0,NaN,NaN,NaN
13,13,11.18034,2.0,0.0,4.879037,0,0,0.0,1,6.87957,...,True,True,0.0,NaN,799,Analysis_1636046826216022929_0_100_Version9,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124285,124285,5.0,2.0,0.0,0.0,21,21,41.983333,1,5.053757,...,True,False,41.983333,NaN,787,Analysis_1636046513186951533_0_23_Version9,21,13873.623188,NaN,NaN
124381,124381,31.048349,2.0,0.0,24.193018,21,21,41.983333,1,8.32431,...,True,False,41.983333,NaN,787,Analysis_1636046513186951533_0_23_Version9,21,13150.144928,NaN,NaN
124411,124411,1.0,2.0,0.0,0.0,21,21,41.983333,1,6.302123,...,True,False,41.983333,NaN,787,Analysis_1636046513186951533_0_23_Version9,21,5382.028986,NaN,NaN
124488,124488,26.925824,2.0,0.0,0.0,21,21,41.983333,1,8.719647,...,True,False,41.983333,NaN,787,Analysis_1636046513186951533_0_23_Version9,21,13110.724638,NaN,NaN


In [7]:
plates = time_plate_info['Plate'].unique()
ind_plate_table= {}
for plate in plates:
    ind_plate_table[plate]=time_plate_info.loc[time_plate_info['Plate']==plate].copy()
    

In [8]:
def get_branch(row):
    if row['Plate']==94:
        hyph = row['end']
        name = f'branches/ts_{hyph}_auto.npy'
        if not os.path.isfile(name):
            return(None)
        else:
            t_real = np.load(name)
            return(row['time_since_emergence'] in t_real)
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]

global_hypha_info['lapse']=global_hypha_info['time_stop_growth']-global_hypha_info['time_init_growth']
time_hypha_info['distance_final_pos_smooth'] = time_hypha_info['distance_final_pos'].rolling(window = 5).mean()
time_hypha_info['new_branch'] = time_hypha_info.apply(get_branch, axis=1)

In [9]:
plates = time_hypha_info['Plate'].unique()
time_hypha_table = {}
for plate in plates: 
    select_time_plate = ind_plate_table[plate]
    last_time = np.max(select_time_plate.loc[select_time_plate['out_study']==0]['time_since_begin'])
#     last_time2 = np.max(select_time_plate.loc[select_time_plate['local_growth_rate']>=0.5]['time_since_begin'])
    select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate].copy()
#     select_time = select_time.loc[select_time['speed'].between(40,400)]
#     select_global = global_hypha_info.loc[global_hypha_info['Plate']==plate]
    select_time = select_time.loc[select_time['has_reached_final_pos']=='False']
    select_time = select_time.loc[select_time['distance_final_pos']>200]
#     select_time = select_time.loc[select_time['in_ROI']=='True']
#     select_time = select_time.loc[select_time['time_since_begin_exp']<=last_time2]
    select_time['speed'] = select_time['speed'].astype(float)
    time_hypha_table[plate] = select_time

In [10]:
roundin = 1000
plates = list(global_hypha_info['Plate'].unique())
plates.remove(10)
ind_hypha_table = {}
for plate in plates:
    table =global_hypha_info.loc[global_hypha_info['Plate']==plate].copy()
    table = table.set_index('hypha')
    ind_hypha_table[plate] = table
for plate in plates:
    table = ind_hypha_table[plate].copy()
    table_time = time_hypha_table[plate]
    select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
    speed_at_init = table_time.loc[table_time['time_since_emergence']==0]
    table['max_speed'] = table_time.groupby('end')['speed'].max()
    table['mean_speed2'] = table_time.groupby('end')['speed'].mean()
    table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
    table['growth_lapse'] = table_time.groupby('end')['speed'].count()
    table = ind_hypha_table[plate]
    table_time = time_hypha_table[plate]
    select_time = time_hypha_info.loc[time_hypha_info['Plate']==plate]
    speed_at_init = table_time.loc[table_time['time_since_emergence']==0]
    table['max_speed'] = table_time.groupby('end')['speed'].max()
    table['mean_speed2'] = table_time.groupby('end')['speed'].mean()
    table['log_length'] = np.log10((table['tot_length_C']+1).astype(float))
    table['log_growth'] = np.log10((table['tot_growth_C']+1).astype(float))

    table['growth_lapse'] = table_time.groupby('end')['speed'].count()
    table['speed_at_init'] = speed_at_init.groupby('end')['speed'].max()
#     table = table.loc[table['max_speed'].notnull()]
    table = table.loc[table['av_width_final'].between(0,15)]
#     table = table.loc[table['num_branch']>=0]
#     table = table.loc[table['tot_growth_C']>=200]

    table['branch_frequ'] = table['num_branch']/(table['tot_growth_C']+1)
# #     table = table.loc[table['branch_frequ'].between(0,0.003)]
#     from sklearn.cluster import KMeans
#     df = table[['av_width_final','log_length','max_speed']].fillna(0)
#     features = ['av_width_final','log_length','max_speed']# Separating out the features
#     # features = ['log_length','mean_speed2']# Separating out the features

#     x = table.loc[:, features].values# Separating out the target
#     # y = table.loc[:,['target']].values# Standardizing the features
#     x = StandardScaler().fit_transform(x)
#     kmeans = KMeans(n_clusters=2).fit(x)
#     centroids = kmeans.cluster_centers_
#     c= kmeans.labels_.astype(float)

#     table['category'] = c
#     table['is_rh'] = (table['category']==table.groupby('category')['max_speed'].mean().idxmax()).astype(int)
#     ind_hypha_table[plate] = table

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [11]:
for plate in plates:
    table = ind_plate_table[plate]
    table['time_since_begin2']=table['time_since_begin']
table = ind_plate_table[799]
table['time_since_begin2']=table['time_since_begin']-150

In [12]:
for plate in plates:
    table = ind_plate_table[plate]
    table['total_length_m'] = table['tot_length_study']/10**6
    table['speed'] = table['tot_length_study'].diff(5)/table['time_since_begin'].diff(5)
    table['mean_step'] = np.mean(table['time_since_begin'].diff(1))
    table['density'] = table['tot_length_study']/(table['area_sep_comp']+1)
    table['ring_density'] = table['tot_length_study'].diff(10)/(table['area_sep_comp'].diff(10)+1)
    table['log_length_study'] = np.log2(table['tot_length_study'].astype(float))
    table['local_growth_rate'] = np.gradient(table['log_length_study'],table['time_since_begin'])*24
    table_hypha = ind_hypha_table[plate]

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\numpy\lib\function_base.py:1080: RuntimeWarning: invalid value encountered in multiply
  out[tuple(slice1)] = a * f[tuple(slice2)] + b * f[tuple(slice3)] + c * f[tuple(slice4)]
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\numpy\lib\function_base.py:1080: RuntimeWarning: invalid value encountered in add
  out[tuple(slice1)] = a * f[tuple(slice2)] + b * f[tuple(slice3)] + c * f[tuple(slice4)]


In [13]:
for plate in plates:
    table = ind_plate_table[plate]
    table_hypha = ind_hypha_table[plate]
    select_time = time_hypha_table[plate]
    df = pd.concat([select_time.groupby('time_since_begin_exp')['speed'].mean()], axis=1).copy()
    df = df.sort_index()
    df = df.reset_index()
    df2 = select_time.groupby('time_since_begin_exp').size()
#     df2 = df.reset_index()
    table['time_since_begin'] = roundin*table['time_since_begin'].astype(int)/roundin 
    df['time_since_begin'] = roundin*df['time_since_begin_exp'].astype(int)/roundin 
    df=df.set_index('time_since_begin')
    table = table.set_index('time_since_begin')
    table = table.sort_index()
    table['mean_speed_hyphae'] = df['speed']
#     table['num_growing_hyphae'] = df2['speed']
    table = table.reset_index()
    ind_plate_table[plate] = table

In [14]:
select_plates = list(global_hypha_info['Plate'].unique())
select_plates.remove(10)
select_plates = [94,792,76,152,102,799]
for plate in select_plates:
    table = ind_plate_table[plate]
    table_hypha = ind_hypha_table[plate]
    select_time = time_hypha_table[plate]
    table_hypha['num_branch']=table_hypha['num_branch'].fillna(-1)
    table_hypha['tot_growth_C']=table_hypha['tot_growth_C'].fillna(0)
    table_hypha['branch_frequ'] = table_hypha['tot_growth_C'].div(table_hypha['num_branch'])/1000
    table_hypha['branch_frequ_time']=table_hypha['branch_frequ']*table_hypha['mean_speed2']
    def get_number_new_branch(row):
        time = row['time_since_begin']
#         growth_num = len(table_hypha.loc[(table_hypha['time_init_growth'].astype(int) == int(time)) &((table_hypha['time_stop_growth'].astype(int) != int(time)))])
        growth_num = len(table_hypha.loc[(table_hypha['time_init_growth'].astype(int) == int(time))])

        return(growth_num)
    def get_number_rh(row):
        time = row['time_since_begin']
        growth_num = len(table_hypha.loc[(table_hypha['time_init_growth'].astype(int) <= int(time)) &((table_hypha['is_rh']==1))])
        return(growth_num)
    def get_number_bas(row):
        time = row['time_since_begin']
        growth_num = len(table_hypha.loc[(table_hypha['time_init_growth'].astype(int) <= int(time)) &((table_hypha['is_rh']==0))])
        return(growth_num)
    def get_number_stop_growth(row):
        time = row['time_since_begin']
        growth_num = len(table_hypha.loc[(table_hypha['time_stop_growth'].astype(int) == int(time)) &((table_hypha['time_init_growth'].astype(int) != int(time)))&((table_hypha['timestep_stop_growth'] != table_hypha['strop_track']))&(table_hypha['out_of_ROI'].isnull())])
        return(growth_num)
    def get_mean_length_hyphae(row):
        time = row['time_since_begin']
        selection = table_hypha.loc[(table_hypha['time_init_growth'].astype(int) == int(time)) &((table_hypha['time_stop_growth'].astype(int) != int(time)))]
        selection = selection.loc[((table_hypha['timestep_stop_growth'] != table_hypha['strop_track']))&(table_hypha['out_of_ROI'].isnull())]
#         selection = selection.loc[selection['is_rh']==1]
#         selection = selection.loc[selection['tot_length_C']<=3200]
        mean_length = np.mean(selection['tot_length_C'])
        return(mean_length)
    def get_mean_speed_hyphae(row):
        time = row['time_since_begin']
        mean_length = np.mean(table_hypha.loc[(table_hypha['time_init_growth'] == time) &((table_hypha['time_stop_growth'] != time))]['mean_speed'])
        return(mean_length)
    def get_mean_growth_lapse_hyphae(row):
        time = row['time_since_begin']
        selection = table_hypha.loc[(table_hypha['time_init_growth'] == time) &((table_hypha['time_stop_growth'] != time))&((table_hypha['timestep_stop_growth'] != table_hypha['strop_track']))&(table_hypha['out_of_ROI'].isnull())]
        mean_lapse = np.mean(selection['time_stop_growth']-selection['time_init_growth'])/24
        return(mean_lapse)
    def get_mean_growth_arrest_frequ(row):
        time = row['time_since_begin']
        selection = table_hypha.loc[(table_hypha['time_init_growth'].astype(int) == int(time)) &((table_hypha['time_stop_growth'].astype(int) != int(time)))&((table_hypha['timestep_stop_growth'] != table_hypha['strop_track']))&(table_hypha['out_of_ROI'].isnull())]
        selection = selection.loc[selection['mean_speed2'].between(150,350)]
#         mean_frequ = np.mean(1/(selection['time_stop_growth']-selection['time_init_growth']))*24
        mean_frequ = np.mean(selection['mean_speed2']/(selection['tot_growth_C']))*24
        return(mean_frequ)
    def get_branch_frequ_micro(row):
        time = row['time_since_begin']
        selection = table_hypha.loc[(table_hypha['time_init_growth'].astype(int) == int(time)) &((table_hypha['time_stop_growth'] != time))&((table_hypha['timestep_stop_growth'] != table_hypha['strop_track']))&(table_hypha['out_of_ROI'].isnull())]
        mean_branch_frequ = np.mean(selection.loc[(selection['branch_frequ']>0)& (selection['branch_frequ']<10)]['branch_frequ'])
        return(mean_branch_frequ)
    def get_branch_frequ_micro_time(row):
        time = row['time_since_begin']
        selection = table_hypha.loc[(table_hypha['time_init_growth'].astype(int) == int(time)) &((table_hypha['time_stop_growth'].astype(int) != int(time)))&((table_hypha['timestep_stop_growth'] != table_hypha['strop_track']))&(table_hypha['out_of_ROI'].isnull())]
        selection = selection.loc[selection['mean_speed2'].between(150,350)]
        mean_branch_frequ = np.mean(selection.loc[(selection['branch_frequ']>0)& (selection['branch_frequ']<10)]['branch_frequ_time'])/1000*24
        return(mean_branch_frequ)
    window = int(1/table['mean_step'].unique()*20)
#     window = 5
    table['num_new_tips'] = table.apply (lambda row: get_number_new_branch(row), axis=1)
#     table['num_rh'] = table.apply (lambda row: get_number_rh(row), axis=1)
#     table['num_bas'] = table.apply (lambda row: get_number_bas(row), axis=1)
#     table['rh_bas_ratio'] = table['num_rh']/table['num_bas']
    table['cum_num_new_tips'] = table['num_new_tips'].cumsum()
#     table['cum_num_new_tips'] = table['num_tips']
    table['branch_frequ'] = table['cum_num_new_tips'].diff(window)/table['tot_length_study'].diff(window)*1000
    table['num_tips_arrest'] = table.apply (lambda row: get_number_stop_growth(row), axis=1)
    table['mean_length_hyphae'] = table.apply (lambda row: get_mean_length_hyphae(row), axis=1)

    table['mean_length_hyphae_smooth'] = table['mean_length_hyphae'].rolling(window=window*2,min_periods=1).mean()
    table['lapse_tip_arrest'] = table.apply (lambda row: get_mean_growth_lapse_hyphae(row), axis=1)
    table['lapse_tip_arrest_smooth'] = table['lapse_tip_arrest'].rolling(window=window,min_periods=1 ).mean()
    table['frequ_tip_arrest'] = table.apply (lambda row: get_mean_growth_arrest_frequ(row), axis=1)
    table['frequ_tip_arrest_smooth'] = table['frequ_tip_arrest'].rolling(window=window,min_periods=1 ).mean()
    table['rate_tip_arrest'] = 1/table['lapse_tip_arrest_smooth']
    table['mean_speed_hyphae'] = table.apply (lambda row: get_mean_speed_hyphae(row), axis=1)
    table['mean_speed_hyphae_smooth'] = table['mean_speed_hyphae'].rolling(window=window,min_periods=1).mean()
    table['branch_frequ_time'] = table['branch_frequ']/1000*table['mean_speed_hyphae_smooth']*24
    table['branch_frequ_micro'] = table.apply (lambda row: get_branch_frequ_micro(row), axis=1)
    table['branch_frequ_micro_smooth'] = table['branch_frequ_micro'].rolling(window=window,min_periods=1).mean()
    table['branch_frequ_micro_time'] = table.apply (lambda row: get_branch_frequ_micro_time(row), axis=1)
#     table['branch_frequ_micro_time_smooth'] = savgol_filter(table['branch_frequ_micro_time'], 2*window//2+1, 3)
    table['branch_frequ_micro_time_smooth'] = table['branch_frequ_micro_time'].rolling(window=window,min_periods=1).mean()
    table['cum_num_tips_arrest'] = table['num_tips_arrest'].cumsum()
    table['log_cum_num_tips_arrest'] = np.log2(table['cum_num_tips_arrest'].astype(float))
    table['local_rate_tip_arrest'] = table['log_cum_num_tips_arrest'].diff(window)/table['time_since_begin'].diff(window)*24
    table['tips_arrest_to_new_length_ratio'] = table['cum_num_tips_arrest'].diff(window)/table['tot_length_study'].diff(window)*1000
    table['log_cum_num_new_tips'] = np.log2(table['cum_num_new_tips'].astype(float))
    table['local_rate_tip_creation'] = table['log_cum_num_new_tips'].diff(window)/table['time_since_begin'].diff(window)*24
    table['local_growth_rate_deduced'] = table['local_rate_tip_creation']-table['rate_tip_arrest']
    table['local_growth_arrest_deduced'] = table['local_growth_rate']-table['local_rate_tip_creation']
    table['alpha_stop'] = table['mean_speed_hyphae_smooth']/table['mean_length_hyphae_smooth']*24
    table['alpha_stop2']= 1/table['lapse_tip_arrest_smooth']

    table['local_growth_rate_deduced_2'] = table['branch_frequ_time']-table['alpha_stop']

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\coren\anaconda3\envs\cleanMsc\lib\si

In [15]:
plt.close('all')
x = 'time_since_begin2'
y = 'branch_frequ'
logy =False
ylim = (0.4,1.2)
xlim = (0,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
# ylabel = f'mean length of RH ($\mu m$)'
ylabel = f'mean length of RH ($\mu m$)'

num_columns = 2
select_plates = [94,792,76,152,102,799]
fig, axs = plt.subplots(((len(select_plates)+1)//num_columns), num_columns)

for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,legend = False,title = plate,logy=logy,ylim=ylim,xlim=xlim,kind = kind)
        #,)
#     if len(out_study)>0:
#         out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'red',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
   
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'mean length of RH ($\\mu m$)')

In [16]:
plt.close('all')
x = 'time_since_begin2'
y = 'num_rh'
logy =False
ylim = (4000,8000)
xlim = (0,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = f'mean length of RH ($\mu m$)'
num_columns =2 
select_plates = [94,792,76,152,102]
fig, axs = plt.subplots(((len(select_plates)+1)//num_columns), num_columns)

for i,plate in enumerate(select_plates):
    print(plate)
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,legend = False,title = plate,logy=logy,ylim=ylim,xlim=xlim,kind = kind)
        #,)
    if len(out_study)>0:
        out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'red',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
    
x = 'time_since_begin2'
y = 'num_bas'  

for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,legend = False,logy=logy,ylim=ylim,xlim=xlim,kind = kind,color = 'purple')
        #,)
    if len(out_study)>0:
        out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'orange',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
# plt.tick_params(labelcolor="none", bottom=False, left=False)
   
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

94


KeyError: 'num_rh'

In [ ]:
plt.close('all')
x = 'time_since_begin2'
y = 'rh_bas_ratio'
logy =False
ylim = (0.4,2)
xlim = (0,150)
# ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = f'total RH/BAS ratio'
num_columns = 2
fig, axs = plt.subplots(((len(select_plates)+1)//num_columns), num_columns)

for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,legend = False,title = plate,logy=logy,ylim=ylim,xlim=xlim,kind = kind)
        #,)
#     if len(out_study)>0:
#         out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'red',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

In [17]:
plt.close('all')
x = 'time_since_begin'
y = 'local_growth_rate'
logy =False
ylim = (100,250)
xlim = (-40,150)
ylim = (0,3)
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Hyphae mean growth speed $(\mu m.h{-1})$'
num_columns = 2
fig, axs = plt.subplots(((len(select_plates)+1)//num_columns), num_columns)
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,legend = False,title = plate,logy=logy,ylim=ylim,xlim=xlim,kind = kind)
        #,)
#     if len(out_study)>0:
#         out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'red',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)

# plt.tick_params(labelcolor="none", bottom=False, left=False)
   
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'Hyphae mean growth speed $(\\mu m.h{-1})$')

In [18]:
plt.close('all')
x = 'time_since_begin2'
y = 'total_length_m'
logy =False
# ylim = (100,300)
xlim = (0,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Hyphae mean growth speed $(\mu m.h{-1})$'
num_columns = 2
fig, axs = plt.subplots(((len(select_plates)+1)//num_columns), num_columns)
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,legend = False,title = plate,logy=logy,ylim=ylim,xlim=xlim,kind = kind)
        #,)
#     if len(out_study)>0:
#         out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'red',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
    ax.set_yscale('log')

# plt.tick_params(labelcolor="none", bottom=False, left=False)
   
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'Hyphae mean growth speed $(\\mu m.h{-1})$')

In [19]:
fig, ax = plt.subplots()
x = 'time_since_begin3'
y = 'total_length_m'
logy =False
# ylim = (100,300)
xlim = (-40,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Total network length $(\mu m)$'
num_columns = 2
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["total_length_m"]>=0.2]["time_since_begin"])
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0
    in_study = table.loc[table['out_study']==0]
    if len(in_study)>0:
        ax.scatter(in_study[x],in_study[y],label = plate)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
    ax.set_yscale('log')

# plt.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_xlim(xlim)  
plt.legend()
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'Total network length $(\\mu m)$')

In [23]:
fig, ax = plt.subplots()
ax2=ax.twinx()
x = 'time_since_begin4'
y = 'total_length_m'
logy =False
# ylim = (100,300)
xlim = (-40,100)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Total network length $(\mu m)$'
num_columns = 2
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["total_length_m"]>=0.2]["time_since_begin"])
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0
    table['time_since_begin4'] = table['time_since_begin']-t1s[plate]
    in_study = table.loc[table['out_study']==0]
    if len(in_study)>0:
        ax.scatter(in_study[x],in_study[y],label = plate)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
    ax.set_yscale('log')
    y = 'total_length_m'
    f = interp1d(in_study[x], in_study[y])
    deb = np.min(in_study[x])
    end = np.max(in_study[x])
    frequ = 1
    av_lapse = 311
    x0 = np.linspace(deb,end,1000)
    resampled = pd.DataFrame({'time':x0,'total_length_m':f(x0)})
    reg = LinearRegression().fit(x0.reshape((-1, 1)), f(x0))
    resampled['total_length_m_smooth']= savgol_filter(resampled['total_length_m'], av_lapse, 3)
    resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
    resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24
    ax2.plot(resampled['time'],resampled['local_growth_rate'],label = plate)

# plt.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_xlim(xlim)  
ax2.set_ylim((0,3))  
ax2.set_ylabel('relative growth rate (.day-1)',size = 20)
plt.legend()
ax2.vlines(0,0,3,color="black")

fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'Total network length $(\\mu m)$')

In [24]:
fig, ax = plt.subplots()
ax2=ax.twinx()
x = 'time_since_begin4'
y = 'total_length_m'
logy =False
# ylim = (100,300)
xlim = (-40,100)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Total network density $(\mu m.mm-2)$'
num_columns = 2
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["total_length_m"]>=0.2]["time_since_begin"])
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0
    table['time_since_begin4'] = table['time_since_begin']-t1s[plate]

    in_study = table.loc[table['out_study']==0]
#     if len(in_study)>0:
#         ax.scatter(in_study[x],in_study[y],label = plate)
#     ax.set_xlabel(None,fontsize=5)
#     ax.set_ylabel(None,fontsize=5)
#     ax.set_yscale('log')
    y = 'total_length_m'
    f = interp1d(in_study[x], in_study[y])
    deb = np.min(in_study[x])
    end = np.max(in_study[x])
    frequ = 1
    av_lapse = 311
    x0 = np.linspace(deb,end,1000)
    resampled = pd.DataFrame({'time':x0,'total_length_m':f(x0)})
    reg = LinearRegression().fit(x0.reshape((-1, 1)), f(x0))
    resampled['total_length_m_smooth']= savgol_filter(resampled['total_length_m'], av_lapse, 3)
    resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
    resampled['local_growth_rate'] = np.gradient(resampled['log_total_length_m_smooth'],resampled['time'])*24
    ax2.plot(resampled['time'],resampled['local_growth_rate'],label = plate)
y = "density"
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["total_length_m"]>=0.2]["time_since_begin"])
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0
    in_study = table.loc[table['out_study']==0]
    if len(in_study)>0:
        ax.scatter(in_study[x],in_study[y],label = plate)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
# plt.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_xlim(xlim)  
ax.set_ylim((0,2000))  

ax2.set_ylim((0,3))  
ax2.set_ylabel('relative growth rate (.day-1)',size = 20)
plt.legend()
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'Total network density $(\\mu m.mm-2)$')

In [22]:
t0s = {94 : 22,102 : 32,152:27,792 : 44,76:13}
t1s = {94 : 46,102 : 60,152:51,792 : 61,76:28}

In [ ]:
table

In [25]:
x = 'time_since_begin4'
y = 'area_sep_comp'
fig, ax = plt.subplots()
ax2=ax.twinx()

delta = -60
for i,plate in enumerate(select_plates):    
    table = ind_plate_table[plate]
    y0 = np.min(table.loc[table['time_since_begin']>=t1s[plate]+delta][y])
    table = table.loc[table['time_since_begin']>=t1s[plate]+delta].copy()
    table[y] = table[y].astype(float)
    table = table.loc[table['out_study']==0].copy()

    table['renorm'] = table[y]-y0+1
    table['log_renorm'] = np.log(table["renorm"])
    table['time_since_begin4'] = table['time_since_begin']-(t1s[plate]+delta)+1
    table['log_time'] = np.log(table["time_since_begin4"])
    table['derivative'] = np.gradient(table["log_renorm"],table['log_time'])

#     table['derivative']
    ax.scatter(table['log_time'],table['log_renorm'],label = plate)
    ax2.plot(table['log_time'],table['derivative'],label = plate)

ax.set_xlim(0)
ax.set_ylim(0)
ax2.set_ylim(0,3)

# ax.plot(table['log_time'],2*table['log_time'])
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
x = 'time_since_begin4'
y = 'area_sep_comp'
fig, ax = plt.subplots()
ax2=ax.twinx()

delta = -60
av_lapse = 11
for i,plate in enumerate(select_plates):    
    table = ind_plate_table[plate]
    y0 = np.min(table.loc[table['time_since_begin']>=t1s[plate]+delta][y])
    table = table.loc[table['time_since_begin']>=t1s[plate]+delta].copy()
    table[y] = table[y].astype(float)
    table = table.loc[table['out_study']==0].copy()

    table['renorm'] = table[y]-y0+1
    table['log_renorm'] = np.log(table["renorm"])
    table['time_since_begin4'] = table['time_since_begin']-(t1s[plate]+delta)+1
    table['log_time'] = np.log(table["time_since_begin4"])
    table['derivative'] = np.gradient(table["log_renorm"],table['log_time'])
    table['derivative'] = savgol_filter(table['derivative'], av_lapse, 1)

#     table['derivative']
    ax.scatter(table['time_since_begin4'],table[y],label = plate)
    ax2.plot(table['time_since_begin4'],table['derivative'],label = plate)
    ax.set_yscale('log')
    ax.set_xscale('log')
ax.set_xlim(0)
ax.set_ylim(0)
ax2.set_ylim(0,4)
ax.set_xlabel('time (h)')
ax.set_ylabel('network length $(mm)$')
ax2.set_ylabel('slope of the log-log')
ax2.vlines(-delta,0,4,color= 'black')

# ax.plot(table['log_time'],2*table['log_time'])
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.


In [ ]:
x = np.linspace(1,10,100)
y = 4*(x+5)**2+150
fig, ax = plt.subplots()

ax.plot(np.log(x+5),np.log(y))
ax.plot(np.log(x+5),np.gradient(np.log(y),np.log(x+5)))

In [ ]:
np.gradient(resampled[y],resampled['time'])/(np.pi/2*0.3**2)

In [27]:
select_plates = [152]
# select_plates = [94,792,76,152,102]

fig, ax = plt.subplots()
ax2=ax.twinx()
x = 'time_since_begin4'
y = 'total_length_m'
y2 = 'derivative'
logy =False
# ylim = (100,300)
xlim = (-20,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Total network area $(mm^2)$'
tau = 50
num_columns = 2
delta = 200
av_lapse = 211

frequ = 0.1
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["total_length_m"]>=0.2]["time_since_begin"])
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0
    table['time_since_begin4'] = table['time_since_begin']-t1s[plate]+tau
    in_study = table.loc[table['out_study']==0]
#     if len(in_study)>0:
#         ax.scatter(in_study[x],in_study[y],label = plate)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
#     ax.set_yscale('log')
#     ax.set_xscale('log')

    f = interp1d(in_study[x], in_study[y])
    deb = np.min(in_study[x])
    end = np.max(in_study[x])
    x0 = np.linspace(deb,end,int((end-deb)/frequ))
    resampled = pd.DataFrame({'time':x0,y:f(x0)})
    reg = LinearRegression().fit(x0.reshape((-1, 1)), f(x0))
    resampled['log_total_length']=np.log2(resampled[y])

    resampled['log_total_length_m_smooth']= savgol_filter(resampled['log_total_length'], av_lapse, 3)
#     resampled['log_time']=np.log2(resampled['time'])
    resampled['time2']=resampled['time']

#     resampled['log_total_length_m_smooth'] = resampled['log_total_length_m_smooth']-np.log(np.min(table.loc[table['time_since_begin']>=t1s[plate]][y]))
#     resampled['derivative'] = resampled[y].diff(delta)/resampled['time'].diff(delta)/(np.pi/2*0.3**2)
    resampled['derivative'] = np.gradient(resampled[y],resampled['time'])/(np.pi/2*0.3**2)
    resampled['derivative'] = savgol_filter(resampled['derivative'], av_lapse, 1)
    resampled['2ndderivative'] = np.gradient(resampled['derivative'],resampled['time'])
#     resampled['2ndderivative'] = savgol_filter(resampled['2ndderivative'], av_lapse, 1)
    ax2.vlines(tau-t0s[plate],0,0.20,color='black')

#     resampled['2ndderivative'] = resampled['derivative'].diff(delta)/resampled['time'].diff(delta)

#     ax2.scatter(resampled['time'],resampled['local_growth_rate'],label = plate)
    in_study = table.loc[table['out_study']==0]
    if len(in_study)>0:
        ax.scatter(in_study[x],in_study[y],label = plate)
    ax2.plot(resampled['time2'],resampled[y2],label = plate)
sat_density = 1000
incr = -0.03
ax2.plot(resampled['time2'],2*resampled['time2']/sat_density+incr,label = "y=2x/sat_density+beta",linestyle = ':')

# plt.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_xlim(xlim)  

ax2.vlines(tau,0,0.20,color='black')
# ax2.set_ylim((0,25))  
# ax2.set_ylim((0,3))  
ax2.set_ylim(None)  

ax2.set_ylabel('derivative rescaled by $\pi / 2 *v^2$',size = 20)
plt.legend()
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.04, 0.5, 'Total network area $(mm^2)$')

In [28]:
select_plates = [94]
fig, ax = plt.subplots()
ax2=ax.twinx()
x = 'time_since_begin3'
y = 'area_sep_comp'
logy =False
# ylim = (100,300)
xlim = (-40,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Total network length $(\mu m)$'
num_columns = 2
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["total_length_m"]>=0.2]["time_since_begin"])
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0
    in_study = table.loc[table['out_study']==0]
#     if len(in_study)>0:
#         ax.scatter(in_study[x],in_study[y],label = plate)
#     ax.set_xlabel(None,fontsize=5)
#     ax.set_ylabel(None,fontsize=5)
#     ax.set_yscale('log')
    f = interp1d(in_study[x], in_study[y])
    deb = np.min(in_study[x])
    end = np.max(in_study[x])
    frequ = 1
    av_lapse = 311
    x0 = np.linspace(deb,end,1000)
    resampled = pd.DataFrame({'time':x0,y:f(x0)})
    reg = LinearRegression().fit(x0.reshape((-1, 1)), f(x0))
    resampled['total_length_m_smooth']= savgol_filter(resampled[y], av_lapse, 3)
    resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
    resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24
    ax.plot(resampled['time'],resampled['local_growth_rate'],label = plate)
x = 'time_since_begin3'
y = 'total_length_m'
logy =False
# ylim = (100,300)
xlim = (-40,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Total network length $(\mu m)$'
num_columns = 2
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
    t0 = np.min(table.loc[table["total_length_m"]>=0.2]["time_since_begin"])
#     ylim = (0,min(np.max(table['speed']),60000))
    table['time_since_begin3'] = table['time_since_begin']-t0
    in_study = table.loc[table['out_study']==0]
#     if len(in_study)>0:
#         ax.scatter(in_study[x],in_study[y],label = plate)
#     ax.set_xlabel(None,fontsize=5)
#     ax.set_ylabel(None,fontsize=5)
#     ax.set_yscale('log')
    f = interp1d(in_study[x], in_study[y])
    deb = np.min(in_study[x])
    end = np.max(in_study[x])
    frequ = 1
    av_lapse = 311
    x0 = np.linspace(deb,end,1000)
    resampled = pd.DataFrame({'time':x0,y:f(x0)})
    reg = LinearRegression().fit(x0.reshape((-1, 1)), f(x0))
    resampled['total_length_m_smooth']= savgol_filter(resampled[y], av_lapse, 3)
    resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
    resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
    resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24
    ax.plot(resampled['time'],resampled['local_growth_rate'],label = plate)
# plt.tick_params(labelcolor="none", bottom=False, left=False)
ax.set_ylim((0,3))  
ax2.set_ylim((0,3))  
ax2.set_ylabel('relative growth rate (.day-1)',size = 20)
plt.legend()
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Text(0.04, 0.5, 'Total network length $(\\mu m)$')

In [29]:
to_plot = ind_plate_table[94]
to_plot = to_plot.loc[to_plot['out_study']==0]

y = 'total_length_m'
f = interp1d(to_plot['time_since_begin'], to_plot[y])
deb = np.min(to_plot['time_since_begin'])
end = np.max(to_plot['time_since_begin'])
frequ = 1
av_lapse = 311
x = np.linspace(deb,end,1000)
resampled = pd.DataFrame({'time':x,'total_length_m':f(x)})
reg = LinearRegression().fit(x.reshape((-1, 1)), f(x))
resampled['total_length_m_smooth']= savgol_filter(resampled['total_length_m'], av_lapse, 3)
resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24

fig, ax = plt.subplots()
ax.plot(resampled['time'],resampled['local_growth_rate'])
ax.plot(table['time_since_begin'],table['local_growth_rate'])

ax.set_ylim((0,2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 2.0)

In [30]:
to_plot = ind_plate_table[94]
to_plot = to_plot.loc[to_plot['out_study']==0]

y = 'total_length_m'
alpha = 1
beta =1000
f = lambda x : x**2+alpha*x+beta
deb = np.min(to_plot['time_since_begin'])
end = np.max(to_plot['time_since_begin'])
frequ = 1
av_lapse = 211
x = np.linspace(deb,end,1000)
resampled = pd.DataFrame({'time':x,'total_length_m':f(x)})
reg = LinearRegression().fit(x.reshape((-1, 1)), f(x))
resampled['total_length_m_smooth']= savgol_filter(resampled['total_length_m'], av_lapse, 3)
resampled['log_total_length_m_smooth']=np.log(resampled['total_length_m_smooth'])
resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24

fig, ax = plt.subplots()
ax.plot(resampled['time'],resampled['local_growth_rate'])
# ax.set_ylim((0,2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
plt.close('all')
x = 'time_since_begin2'
y = 'total_length_m'
logy =False
# ylim = (100,300)
xlim = (0,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'Hyphae mean growth speed $(\mu m.h{-1})$'
num_columns = 2
fig, axs = plt.subplots(((len(select_plates)+1)//num_columns), num_columns)
for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,legend = False,title = plate,logy=logy,ylim=ylim,xlim=xlim,kind = kind)
        #,)
#     if len(out_study)>0:
#         out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'red',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
    ax.set_yscale('log')

# plt.tick_params(labelcolor="none", bottom=False, left=False)
   
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
table['branch_frequ_micro_time_smooth']

In [32]:
plt.close('all')
x = 'time_since_begin2'
y = 'density'
logy =False
# ylim = (0,5)
xlim = (0,150)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = 'branching frequency (.day-1)'
num_columns = 3
select_plates = [94,102,792,76,152,88]
fig, axs = plt.subplots(((len(select_plates)+1)//num_columns), num_columns)

for i,plate in enumerate(select_plates):
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,legend = False,title = plate,logy=logy,ylim=ylim,xlim=xlim,kind = kind)
        #,)
#     if len(out_study)>0:
#         out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'red',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)
    
# x = 'time_since_begin'
# y = 'branch_frequ_micro_time_smooth'  

# for i,plate in enumerate(plates):
#     table = ind_plate_table[plate]
# #     ylim = (0,min(np.max(table['speed']),60000))
#     ax = axs[i//num_columns,i%num_columns]
#     in_study = table.loc[table['out_study']==0]
#     out_study = table.loc[table['out_study']==1]
#     if len(in_study)>0:
#         in_study.plot(x,y, ax = ax,legend = False,logy=logy,ylim=ylim,xlim=xlim,kind = kind,color = 'purple')
#         #,)
#     if len(out_study)>0:
#         out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'orange',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
#     ax.set_xlabel(None,fontsize=5)
#     ax.set_ylabel(None,fontsize=5)
# # plt.tick_params(labelcolor="none", bottom=False, left=False)
   
# fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
# fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
table['num_branch']=table['num_branch'].fillna(-1)
table['tot_growth_C']=table['tot_growth_C'].fillna(0)

In [ ]:
table['branch_frequ'] = table['tot_growth_C'].div(table['num_branch'])/1000

In [ ]:
table['branch_frequ_time']=table['branch_frequ']*table['mean_speed']

In [ ]:
np.mean(table.loc[(table['branch_frequ']>0)& (table['branch_frequ']<np.inf)]['mean_speed'])*24/1000

In [33]:
import matplotlib.pyplot as plt

plt.close('all')
x = 'time_since_begin2'
y = 'frequ_tip_arrest_smooth'
logy =False
# ylim = (0,4)
xlim = (0,400)
ylim = None
kind = "scatter"
# fig = plt.figure()
# ax = plt.subplot(111)
xlabel = 'time (h)'
ylabel = r'$\alpha_{stop}(day^{-1})$'
num_columns = 4
fig, axs = plt.subplots(((len(plates)+1)//num_columns), num_columns)
for i,plate in enumerate(plates):
    table = ind_plate_table[plate]
#     ylim = (0,min(np.max(table['speed']),60000))
    ax = axs[i//num_columns,i%num_columns]
    in_study = table.loc[table['out_study']==0]
    out_study = table.loc[table['out_study']==1]
    if len(in_study)>0:
        in_study.plot(x,y, ax = ax,legend = False,title = plate,logy=logy,ylim=ylim,xlim=xlim,kind = kind)
        #,)
    if len(out_study)>0:
        out_study.plot(x,y, ax = ax,legend = False,title = plate,color = 'red',logy=logy,ylim = ylim,xlim=xlim,kind=kind)
    ax.set_xlabel(None,fontsize=5)
    ax.set_ylabel(None,fontsize=5)

# plt.tick_params(labelcolor="none", bottom=False, left=False)
   
fig.text(0.5, 0.004, xlabel, ha='center',fontdict = {'size' : 20} )
fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical',fontdict = {'size' : 20})


# plt.xlabel("Common X-Axis")

# plt.ylabel("Common Y-Axis")
# plt.tight_layout(pad=0.)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'frequ_tip_arrest_smooth'

In [ ]:
plate = 94
table = ind_plate_table[plate]
table_hypha = ind_hypha_table[plate]
selection = table_hypha.loc[(table_hypha['time_init_growth'] !=table_hypha['time_stop_growth'])&((table_hypha['timestep_stop_growth'] != table_hypha['strop_track']))&(table_hypha['out_of_ROI'].isnull())&(table_hypha['mean_speed']<=400)&(table_hypha['num_branch']>0)]


In [ ]:
np.mean(table_hypha['mean_speed'])

In [ ]:
column = 'mean_speed'
kind = "hist"
plt.close("all")
num_columns = 4
fig, axs = plt.subplots(((len(plates)+1)//num_columns), num_columns)
ylim = None
xlim = (0,400)

for i,plate in enumerate(plates):
    ax = axs[i//num_columns,i%num_columns]
    table = ind_plate_table[plate]
    table_hypha = ind_hypha_table[plate]
    selection = table_hypha.loc[(table_hypha['time_init_growth'] !=table_hypha['time_stop_growth'])&((table_hypha['timestep_stop_growth'] != table_hypha['strop_track']))&(table_hypha['out_of_ROI'].isnull())&(table_hypha['mean_speed']<=400)&(table_hypha['num_branch']>0)]
    selection[column].hist(bins = 15,density=True,ax=ax)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title(plate)

In [ ]:
plt.close("all")
for plate in plates:
    table = ind_plate_table[plate]
    table_hypha = ind_hypha_table[plate]
    selection = table_hypha.loc[(table_hypha['time_init_growth'] !=table_hypha['time_stop_growth'])&((table_hypha['timestep_stop_growth'] != table_hypha['strop_track']))&(table_hypha['out_of_ROI'].isnull())]
    lapse_growth = selection['time_stop_growth']-selection['time_init_growth']
    lapse_growth/=24
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.hist(lapse_growth,20)

In [ ]:
table_hypha = ind_hypha_table[94]
series = table_hypha.loc[table_hypha['timestep_anastomosis'].isnull()==False].groupby('timestep_anastomosis')['av_width_final'].count()
fig, ax = plt.subplots()
ax.plot(np.cumsum(series[:-2]))

In [ ]:
series.to_csv("series.csv")

In [ ]:
table_hypha